<a href="https://colab.research.google.com/github/titsitits/UNamur_Python_Analytics/blob/master/7_Machine_Learning_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mickaël Tits
CETIC
mickael.tits@cetic.be

# Introduction au machine learning

Un sous-domaine important de l'analyse de données est le machine learning (apprentissage automatique). Le machine learning regroupe un ensemble de techniques permettant, à partir des données, d'apprendre automatiquement différents types de relations entre les variables.

On distingue généralement:
* Le machine learning **supervisé**, qui permet d'apprendre des relations entre les données et des labels. Plus spécifiquement, on entraîne généralement un algorithme à prédire les labels qui ont été donnés pour un ensemble de données. Il existe deux applications principales de l'apprentissage supervisé:
  * La classification: on a un ensemble de données à partir desquelles on aimerait prédire une classe: le **label** de chaque sample est par exemple "chat" ou chien". Les données permettant d'entraîner l'algorithme sont par exemple les pixels d'images qui ont été labelisées (i.e. on note pour chaque image si elle contient un chat ou un chien). En entrée de l'algorithme, on utilise les images, et en sortie on vérifie la prédiction donnée par l'algorithme. Pour une image contenant un chat, il doit prédire le bon label, c'est-à-dire "chat". L'étape de l'entraînement de l'algorithme consiste alors à **minimiser l'erreur moyenne des prédictions en modifiant itérativement les paramètres de l'algorithme**.
  * Le régression: le **label** à prédire à partir des données est une valeur continue: par exemple le prix d'une maison, à partir de différentes caractéristiques, ou l'âge d'une personne à partir d'une image. Les algorithmes utilisés sont donc différents puisqu'ils voient prédire une valeur continue et non une classe. Cependant, l'entraînement de l'algorithme consiste toujours en une minimisation de l'erreur moyenne des prédictions sur toutes les données d'entraînement.

* Le machine learning **non-supervisé**, qui permet d'apprendre des relations entre différentes données, telles que les similitudes entre différentes variables, ou différentes données (e.g.: Clustering, Réduction de dimensions).


Chargez d'abord le dataframe préparé lors du chapitre précédent (relancer le notebook précédent si nécessaire).

In [0]:
df = pd.read_csv("houses_features.csv", index_col = 0)
dfbxl = df[df.city == "Bruxelles"]

# Prédiction du prix d'une maison

## Préparation du dataset

In [83]:
#L'adresse n'a pas d'intérêt (on a déjà extrait la ville, qui est une catégorie intéressante)
dataset = dfbxl.drop("address",axis=1)

dataset = pd.get_dummies(dataset, drop_first=True)
dataset

,price,price_per_m2,price_per_room,rooms,surface,website_immoweb
2,400000.0,3333.333333,80000.000000,5.0,120,1
7,280000.0,2333.333333,93333.333333,3.0,120,1
8,400000.0,2666.666667,100000.000000,4.0,150,0
9,480000.0,2400.000000,96000.000000,5.0,200,0
10,280000.0,2333.333333,93333.333333,3.0,120,0
11,400000.0,2666.666667,100000.000000,4.0,150,0
12,480000.0,2400.000000,96000.000000,5.0,200,0


In [107]:
dataset.corr()

,price,price_per_m2,price_per_room,rooms,surface,website_immoweb
price,1.000000,0.176690,0.145297,0.925514,0.866031,-0.402939
price_per_m2,0.176690,1.000000,-0.679186,0.453582,-0.337120,0.463636
price_per_room,0.145297,-0.679186,1.000000,-0.238949,0.451827,-0.747105
rooms,0.925514,0.453582,-0.238949,1.000000,0.665445,-0.108465
surface,0.866031,-0.337120,0.451827,0.665445,1.000000,-0.599876
website_immoweb,-0.402939,0.463636,-0.747105,-0.108465,-0.599876,1.000000


## Une librairie Python de référence pour le machine learing: Scikit-learn

Une librairie Python incontournable pour le machine learning est la librairie Scikit-learn: https://scikit-learn.org/stable/ .

Elle inclut un geand nombre d'algorithmes de machine learning supervisé, non-supervisé, de préprocessing de caractéristiques, et de sélection de modèles de machine learning. L'API est assez simple, et est basée sur les librairies vues précédemment (Numpy, Scipy, Matplotlib).

La librairie contient de nombreuses classes permettant d'instancier des modèles de machine learning, que l'on peut ensuite utiliser pour: 
1. Entraîner sur un ensemble de **données d'entraînement (ou training set)**.

2. Chaque modèle contient généralement différents paramètres qui peuvent être adaptés aux données utilisées, afin d'améliorer les performances de l'algorithme. Pour vérifier les performances du modèles selon différents paramètres, on teste le modèle sur un set de **données de validation (ou validation set)**. On parle généralement de processus de **cross-validation**.

3. Afin de vérifier que le modèle obtenu est efficace, on peut ensuite le tester avec de nouvelles données (autres que celles utilisées durant l'entraînement et la cross-validation), les **données de test (ou test set)**.

### Régression linéaire

Etant donné le peu de données de l'exemple, on utilisera un modèle simple (régression linéaire), avec un nombre restreint de caractéristiques (surface et rooms). On ne contentera d'un training set et d'un test set (pas de validation ici: on gardera les paramètres par défaut du modèle).

In [0]:
from sklearn.linear_model import LinearRegression

#On crée un objet de la classe LinearRegression
regressor = LinearRegression()


In [0]:
trainset = dataset.iloc[:6]
testset = dataset.iloc[6:]

Xtrain, ytrain, Xtest, ytest = trainset[["surface","rooms"]], trainset["price"], testset[["surface","rooms"]], testset["price"]

In [91]:
regressor.fit(Xtrain, ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [93]:
print(regressor.predict(Xtest)[0], ytest.values[0])

494399.9999999999 480000.0


In [94]:
regressor.coef_

array([ 1120., 57600.])

## Leave-one-out validation
Lorsque l'ensemble de données est petit (comme dans cet exemple), il est intéressant d'utiliser une technique de tournante sur les données de test pour vérifier la qualité de l'algorithme choisi. Sur l'ensemble des données, on retient itérativement un sous-ensemble différent pour les données de test et on entraîne sur le reste. On peut parcourir itérativement toutes les données, pour ainsi tester au final l'algorithme sur l'ensemble des données.

In [128]:
from sklearn.metrics import mean_squared_error
import numpy as np

testpredictions = []

input_features = ["surface","rooms"]


for i in dataset.index:
  trainset = dataset[dataset.index!=i]
  testset = dataset.loc[i].to_frame().transpose()
  Xtrain, ytrain, Xtest, ytest = trainset[input_features], trainset["price"], testset[input_features], testset["price"]
  regressor.fit(Xtrain, ytrain)
  
  trainpred = regressor.predict(Xtrain)
  testpred = regressor.predict(Xtest)
  
  print(i, " - training mean error:", np.mean(np.abs(trainpred - ytrain)), "\n\t test mean error:", np.mean(np.abs(testpred - ytest)), "\n")
        
  testpredictions.append(testpred[0])
  

2  - training mean error: 1.9402553637822468e-11 
	 test mean error: 240000.0 

7  - training mean error: 11962.616822429933 
	 test mean error: 17943.925233644957 

8  - training mean error: 8258.06451612901 
	 test mean error: 24774.19354838709 

9  - training mean error: 12799.99999999998 
	 test mean error: 14399.999999999942 

10  - training mean error: 11962.616822429904 
	 test mean error: 17943.9252336449 

11  - training mean error: 8258.06451612901 
	 test mean error: 24774.19354838709 

12  - training mean error: 12799.99999999998 
	 test mean error: 14399.999999999884 



In [97]:
testpredictions

[640000.0,
 297943.92523364496,
 375225.8064516129,
 494399.99999999994,
 297943.9252336449,
 375225.8064516129,
 494399.9999999999]

In [109]:
results = dfbxl
results["predictions"] = testpredictions
results

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,address,city,price,price_per_m2,price_per_room,rooms,surface,website,predictions
2,"Porte de Namur 25, Bruxelles",Bruxelles,400000.0,3333.333333,80000.000000,5.0,120,immoweb,298367.346939
7,"Rue de la Loi 51, Bruxelles",Bruxelles,280000.0,2333.333333,93333.333333,3.0,120,immoweb,343061.224490
8,"Rue de la Loi 52, Bruxelles",Bruxelles,400000.0,2666.666667,100000.000000,4.0,150,immovlan,383340.563991
9,"Rue de la Loi 53, Bruxelles",Bruxelles,480000.0,2400.000000,96000.000000,5.0,200,immovlan,489729.729730
10,"Rue de la Loi 51, Bruxelles",Bruxelles,280000.0,2333.333333,93333.333333,3.0,120,immovlan,343061.224490
11,"Rue de la Loi 52, Bruxelles",Bruxelles,400000.0,2666.666667,100000.000000,4.0,150,immovlan,383340.563991
12,"Rue de la Loi 53, Bruxelles",Bruxelles,480000.0,2400.000000,96000.000000,5.0,200,immovlan,489729.729730


In [110]:
results.corr()

,price,price_per_m2,price_per_room,rooms,surface,predictions
price,1.000000,0.176690,0.145297,0.925514,0.866031,0.745566
price_per_m2,0.176690,1.000000,-0.679186,0.453582,-0.337120,-0.522597
price_per_room,0.145297,-0.679186,1.000000,-0.238949,0.451827,0.552488
rooms,0.925514,0.453582,-0.238949,1.000000,0.665445,0.505245
surface,0.866031,-0.337120,0.451827,0.665445,1.000000,0.978828
predictions,0.745566,-0.522597,0.552488,0.505245,0.978828,1.000000


Si les performances sur le test set sont bien plus mauvaises que sur les données d'entraînement, il est fort probable qu'on soit dans une situation d'**overfitting**: le modèle a appris "par coeur" ce qu'il fallait prédire pour chaque donnée du training set. Il ne se généralise donc pas bien sur de nouvelles données. Afin d'éviter ce problème, on peut soit:

* diminuer la complexité (le nombre de degrés de liberté, i.e. de paramètres) du modèle, soit en diminuant le nombre de caractéristique, soit en utilisant un algorithme plus simple.
* entraîner le modèle sur plus de données.

Si à l'inverse, les performances obtenues lors de l'entraînement sont faibles, on est en situation d'**underfitting**. Il faut alors au contraire augmenter la complexité au modèle et extraire des caractéristiques de meilleure qualité (ayant une relation pus forte avec le label à prédire).
